# Tempo Introduction

![architecture](architecture.png)

In this introduction we will:

  * [Describe the project structure](#Project-Structure)
  * [Train some models](#Train-Models)
  * [Create Tempo artifacts](#Create-Tempo-Artifacts)
  * [Run unit tests](#Unit-Tests)
  * [Save python environment for our classifier](#Save-Classifier-Environment)
  * [Test Locally on Docker](#Test-Locally-on-Docker)

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

## Project Structure

In [23]:
!tree -P "*.py"  -I "__init__.py|__pycache__" -L 2

.
├── artifacts
│   ├── classifier
│   ├── sklearn
│   └── xgboost
├── k8s
│   └── rbac
├── src
│   ├── data.py
│   ├── tempo.py
│   └── train.py
└── tests
    └── test_tempo.py

8 directories, 4 files


## Train Models

 * This section is where as a data scientist you do your work of training models and creating artfacts.
 * For this example we train sklearn and xgboost classification models for the iris dataset.

In [1]:
import os
from tempo.utils import logger
import logging
import numpy as np
logger.setLevel(logging.ERROR)
logging.basicConfig(level=logging.ERROR)
ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"

In [ ]:
# %load src/train.py
import joblib
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from src.data import IrisData

SKLearnFolder = "sklearn"
XGBoostFolder = "xgboost"


def train_sklearn(data: IrisData, artifacts_folder: str):
    logreg = LogisticRegression(C=1e5)
    logreg.fit(data.X, data.y)
    with open(f"{artifacts_folder}/{SKLearnFolder}/model.joblib", "wb") as f:
        joblib.dump(logreg, f)


def train_xgboost(data: IrisData, artifacts_folder: str):
    clf = XGBClassifier()
    clf.fit(data.X, data.y)
    clf.save_model(f"{artifacts_folder}/{XGBoostFolder}/model.bst")


In [3]:
from src.data import IrisData
from src.train import train_sklearn, train_xgboost
data = IrisData()
train_sklearn(data, ARTIFACTS_FOLDER)
train_xgboost(data, ARTIFACTS_FOLDER)

[18:09:05] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/clive/anaconda3/envs/tempo-examples/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Create Tempo Artifacts

 * Here we create the Tempo models and orchestration Pipeline for our final service using our models.
 * For illustration the final service will call the sklearn model and based on the result will decide to return that prediction or call the xgboost model and return that prediction instead.

In [4]:
from src.tempo import get_tempo_artifacts
classifier, sklearn_model, xgboost_model = get_tempo_artifacts(ARTIFACTS_FOLDER)

In [ ]:
# %load src/tempo.py
from typing import Tuple

import numpy as np

from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model
from tempo.serve.pipeline import Pipeline, PipelineModels
from tempo.serve.utils import pipeline

from .train import SKLearnFolder, XGBoostFolder

PipelineFolder = "classifier"
SKLearnTag = "sklearn prediction"
XGBoostTag = "xgboost prediction"


def get_tempo_artifacts(artifacts_folder: str) -> Tuple[Pipeline, Model, Model]:

    sklearn_model = Model(
        name="test-iris-sklearn",
        platform=ModelFramework.SKLearn,
        local_folder=f"{artifacts_folder}/{SKLearnFolder}",
        uri="s3://tempo/basic/sklearn",
    )

    xgboost_model = Model(
        name="test-iris-xgboost",
        platform=ModelFramework.XGBoost,
        local_folder=f"{artifacts_folder}/{XGBoostFolder}",
        uri="s3://tempo/basic/xgboost",
    )

    @pipeline(
        name="classifier",
        uri="s3://tempo/basic/pipeline",
        local_folder=f"{artifacts_folder}/{PipelineFolder}",
        models=PipelineModels(sklearn=sklearn_model, xgboost=xgboost_model),
    )
    def classifier(payload: np.ndarray) -> Tuple[np.ndarray, str]:
        res1 = classifier.models.sklearn(input=payload)

        if res1[0] == 1:
            return res1, SKLearnTag
        else:
            return classifier.models.xgboost(input=payload), XGBoostTag

    return classifier, sklearn_model, xgboost_model


## Unit Tests

 * Here we run our unit tests to ensure the orchestration works before running on the actual models.

In [ ]:
# %load tests/test_tempo.py
import numpy as np
from src.tempo import SKLearnTag, XGBoostTag, get_tempo_artifacts


def test_sklearn_model_used():
    classifier, _, _ = get_tempo_artifacts("")
    classifier.models.sklearn = lambda input: np.array([[1]])
    res, tag = classifier(np.array([[1, 2, 3, 4]]))
    assert res[0][0] == 1
    assert tag == SKLearnTag


def test_xgboost_model_used():
    classifier, _, _ = get_tempo_artifacts("")
    classifier.models.sklearn = lambda input: np.array([[0.2]])
    classifier.models.xgboost = lambda input: np.array([[0.1]])
    res, tag = classifier(np.array([[1, 2, 3, 4]]))
    assert res[0][0] == 0.1
    assert tag == XGBoostTag


In [7]:
!python -m pytest tests/

============================= test session starts ==============================
platform linux -- Python 3.7.9, pytest-6.2.0, py-1.10.0, pluggy-0.13.1
rootdir: /home/clive/work/mlops/fork-tempo, configfile: setup.cfg
plugins: asyncio-0.14.0
collected 2 items                                                              

tests/test_tempo.py ..                                                   [100%]

============================== 2 passed in 1.03s ===============================


## Save Classifier Environment

 * In preparation for running our models we save the Python environment needed for the orchestration to run as defined by a `conda.yaml` in our project.

In [8]:
!cat artifacts/classifier/conda.yaml

name: tempo
channels:
  - defaults
dependencies:
  - python=3.7.9
  - pip:
    - mlops-tempo @ file:///home/clive/work/mlops/fork-tempo
    - mlserver==0.3.1.dev7


In [9]:
from tempo.serve.loader import save
save(classifier)

Packing environment at '/home/clive/anaconda3/envs/tempo-6dae3aec-6d42-4871-a1cf-2816476a992c' to '/home/clive/work/mlops/fork-tempo/docs/examples/intro/artifacts/classifier/environment.tar.gz'
[########################################] | 100% Completed | 12.2s


## Test Locally on Docker

 * Here we test our models using production images but running locally on Docker. This allows us to ensure the final production deployed model will behave as expected when deployed.

In [10]:
from tempo.seldon.docker import SeldonDockerRuntime
docker_runtime = SeldonDockerRuntime()
docker_runtime.deploy(classifier)
docker_runtime.wait_ready(classifier)

In [11]:
classifier(np.array([[1, 2, 3, 4]]))

(array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'xgboost prediction')

In [12]:
print(classifier.remote(np.array([[0, 0, 0,0]])))
print(classifier.remote(np.array([[5.964,4.006,2.081,1.031]])))

{'output0': array([1.], dtype=float32), 'output1': 'sklearn prediction'}
{'output0': array([[0.97329617, 0.02412145, 0.00258233]], dtype=float32), 'output1': 'xgboost prediction'}


In [13]:
docker_runtime.undeploy(classifier)

## Production Option 1 (Deploy to Kubernetes with Tempo)

 * Here we illustrate how to run the final models in "production" on Kubernetes by using Tempo to deploy
 
 ### Prerequisites
 
 Create a Kind Kubernetes cluster with Minio and Seldon Core installed using Ansible from the Tempo project Ansible playbook.
 
 ```
 ansible-playbook ansible/playbooks/default.yaml
 ```

In [14]:
!kubectl apply -f k8s/rbac -n production

secret/minio-secret created
serviceaccount/tempo-pipeline created
role.rbac.authorization.k8s.io/tempo-pipeline created
rolebinding.rbac.authorization.k8s.io/tempo-pipeline-rolebinding created


In [15]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [16]:
from tempo.serve.loader import upload
upload(sklearn_model)
upload(xgboost_model)
upload(classifier)

In [17]:
from tempo.serve.metadata import RuntimeOptions, KubernetesOptions
runtime_options = RuntimeOptions(
        k8s_options=KubernetesOptions(
            namespace="production",
            authSecretName="minio-secret"
        )
    )

In [18]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
k8s_runtime = SeldonKubernetesRuntime(runtime_options)
k8s_runtime.deploy(classifier)
k8s_runtime.wait_ready(classifier)

In [19]:
print(classifier.remote(payload=np.array([[0, 0, 0, 0]])))
print(classifier.remote(payload=np.array([[1, 2, 3, 4]])))

{'output0': array([1.], dtype=float32), 'output1': 'sklearn prediction'}
{'output0': array([[0.00847207, 0.03168794, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}


In [20]:
k8s_runtime.undeploy(classifier)

## Production Option 2 (Gitops)

 * We create yaml to provide to our DevOps team to deploy to a production cluster
 * We add Kustomize patches to modify the base Kubernetes yaml created by Tempo

In [21]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.metadata import RuntimeOptions, KubernetesOptions
runtime_options = RuntimeOptions(
        k8s_options=KubernetesOptions(
            namespace="production",
            authSecretName="minio-secret"
        )
    )
k8s_runtime = SeldonKubernetesRuntime()
yaml_str = k8s_runtime.to_k8s_yaml(classifier)
with open(os.getcwd()+"/k8s/tempo.yaml","w") as f:
    f.write(yaml_str)

In [22]:
!kustomize build k8s

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: classifier
  namespace: production
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - args: []
          env:
          - name: MLSERVER_HTTP_PORT
            value: "9000"
          - name: MLSERVER_GRPC_PORT
            value: "9500"
          - name: MLSERVER_MODEL_IMPLEMENTATION
            value: tempo.mlserver.InferenceRuntime
          - name: MLSERVER_MODEL_NAME
            value: classifier
          - name: MLSERVER_MODEL_URI
            value: /mnt/models
          - name: TEMPO_RUNTIME_OPTIONS
            value: '{"runtime": null, "docker_options": {"defaultRuntime": "tempo.seldon.SeldonDockerRuntime"},
              "k8s_options": {"replicas": 1, "minReplicas": null, "maxReplicas": null,
              "authSecretName": null, "serviceAccountName": null, "defaultRuntime":
              "tempo.seldon.SeldonKubernetesRuntime", "namespace": "default"}, "ingress_optio